In [1]:
import os
import sys
import uuid
import getpass
import pandas as pd
import geopandas as gpd

user = getpass.getuser()

DVUTILS_LOCAL_CLONE_PATH = f"/Users/{user}/Documents/GitHub/dvutils"
sys.path.insert(0, DVUTILS_LOCAL_CLONE_PATH)
from utils_io import *

In [2]:
# set working directory
dir = os.path.join("/Users", user, "Library", "CloudStorage", "Box-Box", "_GIS Analyses", "Spatial Data")

In [3]:
# create a function to remove z values from a geometry
def remove_z(geom):
    """
    Removes z values from a geometry

    Source: https://gist.github.com/rmania/8c88377a5c902dfbc134795a7af538d8?permalink_comment_id=2893099#gistcomment-2893099
    """
    import shapely
    return shapely.wkb.loads(shapely.wkb.dumps(geom, output_dimension=2))

In [7]:
# create a function to read kml files by geometry type and return a geodataframe
def read_kml_by_geom_type(directory, geom_type):
    """Read a kml file by geometry type and return a geodataframe

    Parameters:
    -----------
    path : str
        path to the kml directory
    geom_type : str
        one of ['Point', 'LineString', 'Polygon']
    """
    import re
    import glob
    import fiona
    fiona.supported_drivers['KML'] = 'rw'

    # use glob to get all the csv files
    # in the folder
    if geom_type == "Point":
        type_string = "_point"
    elif geom_type == "LineString":
        type_string = "_segment"
    else:
        type_string = "_polygon"

    pattern = re.compile(type_string)
    file_list = glob.glob(os.path.join(directory, "*.kml"))
    gdfs = list()

    for file in file_list:
        file_name = file.split("/")[-1]
        if pattern.search(file_name):
            # read the file
            gdf = gpd.read_file(file, driver="KML")
            # add the source file name
            gdf["source_file"] = file_name
            # remove z values
            gdf["geometry"] = gdf["geometry"].apply(remove_z)
            gdfs.append(gdf)
    gdf = pd.concat(gdfs, ignore_index=True)
    return gdf

In [8]:
gdf = read_kml_by_geom_type(dir, "Point")

In [5]:
# 1a_bus-delay-congestion_segment.kml seems to have been exported incorrectly; this file contains polygons instead of lines/segments
# 1c_bus-delay-stop-location_segment.kml seems to have been exported incorrectly; this file contains points instead of lines/segments
# 

In [9]:
gdf

,Name,Description,geometry,source_file
0,(2) Shattuck/University,"bus stop violations, delays, accessibility iss...",POINT (-122.26842 37.87212),4_other_point.kml
1,(69) Mission/John Daly,transit vehicles bunching at intersection; act...,POINT (-122.46235 37.70497),1b_bus-delay-signals_point.kml
2,(70) Mission/San Pedro,transit vehicles bunching at intersection; act...,POINT (-122.46592 37.68960),1b_bus-delay-signals_point.kml
3,(75) SR-92/US-101,"heavy backups, especially on weekends; queue j...",POINT (-122.33894 37.50702),1b_bus-delay-signals_point.kml
4,(76) SR-92/Main,"heavy backups, especially on weekends; queue j...",POINT (-122.43013 37.46790),1b_bus-delay-signals_point.kml
5,(89) Old Redwood/Lakewood,general congestion/delay; need for additional ...,POINT (-122.80683 38.54651),1b_bus-delay-signals_point.kml
6,(94) Whipple/Central,EB: bus gets stuck in traffic in through lane....,POINT (-122.03785 37.60595),1b_bus-delay-signals_point.kml
7,(95) Whipple/Railroad,EB: bus gets stuck waiting to merge into traff...,POINT (-122.03130 37.60595),1b_bus-delay-signals_point.kml
8,(100) Samaritan/Union,queuing from SR-85 ramps add 1-2min delay to t...,POINT (-121.93133 37.25052),1b_bus-delay-signals_point.kml
9,(101) Los Gatos/Samaritan,queuing from SR-85 ramps add 1-2 min delay to ...,POINT (-121.95249 37.25317),1b_bus-delay-signals_point.kml
